In [1]:
from os import listdir
from os.path import join, isfile
from lxml import etree
from src.data import PackageManager
from src.data import XbrlConst
from collections import defaultdict
from src.data import xbrl2rdf, process_linkbase, process_instance
import src.data

In [2]:
XBRL_TAXONOMY_PATH = join('..', 'data', 'external', 'taxonomies')
output_dir = join("..", "data", "raw")

taxonomies = [f for f in listdir(XBRL_TAXONOMY_PATH) if isfile(join(XBRL_TAXONOMY_PATH, f)) and f[-3:]=='zip']

manager = PackageManager.Taxonomies(XBRL_TAXONOMY_PATH)

for taxonomy in taxonomies:
    manager.addPackage(join(XBRL_TAXONOMY_PATH, taxonomy))
manager.rebuildRemappings()
manager.save()
for item in manager.config['packages']:
    print(item['URL'])

Packages overlap the same rewrite start string http://www.dnb.nl/
Packages overlap the same rewrite start string http://www.dnb.nl/
Packages overlap the same rewrite start string http://www.dnb.nl/
Packages overlap the same rewrite start string http://www.dnb.nl/
Packages overlap the same rewrite start string http://www.dnb.nl/
Packages overlap the same rewrite start string http://www.dnb.nl/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Packages overlap the same rewrite start string http://www.eurofiling.info/
Pack

In [3]:
f = src.data.openFileSource(manager.config['packages'][2]['URL'])
f.mappedPaths = manager.config['packages'][1]["remappings"]

In [4]:
xmlfiles = [file for file in f.dir if (file[-3:]=='xml')]
print(len(xmlfiles))

30603


In [5]:
roots = list()

for file in xmlfiles:
    fp = f.fs.open(file, "r")
    content = fp.read()
    roots.append(etree.fromstring(content, parser=etree.XMLParser(remove_comments=True)))

In [6]:
namespaces = dict()
for idx, root in enumerate(roots):
    namespaces = {**namespaces, **root.nsmap}

In [7]:
params = {'linkNumber' : 0, 
          'namespaces': namespaces}
s = '' 
for idx, root in enumerate(roots):
    params['base'] = xmlfiles[idx]
    params = process_linkbase.processLinkBase(root, params)
    s += params['output'].getvalue()

In [8]:
with open(join(output_dir, 'taxo.ttl'), 'w') as fp:
    fp.write(s.replace('\u2264',''))